In [1]:
import pandas as pd
import os
import regex as re

# Path to stored files
path = "../Corpus"
corp_files = os.listdir(path)
corp_files

['eco_dorzeczy.csv',
 'eco_gpc.csv',
 'eco_newsweek.csv',
 'eco_polityka.csv',
 'eco_rzepa.csv',
 'eco_wpolityce.csv',
 'eco_wprost.csv',
 'eco_wyborcza_colocates.csv',
 'eco_wyborcza.csv']

## Load and concatenate data

In [4]:
data_list = []
cols = ["text", "id", 'date', 'source']

for file in corp_files:
    # Load the data
    data = pd.read_csv(path+"/"+file, parse_dates=['date'])
   
    # Make sure to only include columns of interest
    if 'id' not in data.columns:
        print(f"No id in {file}")
        data['id'] = data.index
    data = data[cols]   

    # Append to list
    data_list.append(data)

df = pd.concat(data_list)
df = df[df['text'].isna() == False]

No id in eco_gpc.csv
No id in eco_wyborcza.csv


## Preprocessing

In [5]:
df['text_processed'] = df['text'].apply(lambda x: re.sub(r'[\n\.\-]', ' ', x))
df['text_processed'] = df['text_processed'].apply(lambda x: re.sub(r'\s+', ' ', x))
df['text_processed'] = df['text_processed'].apply(lambda x: re.sub(r'[^\dAaĄąBbCcĆćDdEeĘęFfGgHhIiJjKkLlŁłMmNnŃńOoÓóPpRrSsŚśTtUuWwYyZzŹźŻż ]', '', x))
df['text_processed'] = df['text_processed'].str.lower()

## Extracting collocates

In [6]:
word_regex = r'(?:[AaĄąBbCcĆćDdEeĘęFfGgHhIiJjKkLlŁłMmNnŃńOoÓóPpRrSsŚśTtUuWwYyZzŹźŻż\d]+\s+)'
regex_expression = word_regex + r'{1,5}' + "zmian[^ ]* klimat[^ ]*\s+" + word_regex + r'{1,5}'

df['raw_colocates'] = df['text_processed'].apply(lambda x: re.findall(regex_expression, x))

In [8]:
df['raw_colocates']

0                                                      []
1                                                      []
2       [twierdzą  niezbędne żeby przeciwstawić się zm...
3                                                      []
4       [fałszywy alarm jak panika wokół zmiany klimat...
                              ...                        
7223                                                   []
7224                                                   []
7225                                                   []
7226                                                   []
7227                                                   []
Name: raw_colocates, Length: 26642, dtype: object

## Checking the extraction process

In [9]:
df['count'] = df['raw_colocates'].apply(lambda x: len(x))
print(df['count'].sum())

11956


In [10]:
df['real_count'] = df['text_processed'].apply(lambda x: len(re.findall("zmian[^ ]* klimat[^ ]*", x)))
print(df['real_count'].sum())

12162


## Collocates by source

In [11]:
df[df['real_count'] > 0].source.value_counts()

Wyborcza                    4025
Rzeczpospolita              1376
POLITYKA                     255
Gazeta Polska Codziennie     252
Newsweek                     115
wPolityce                     91
WPROST                        69
DoRzeczy                      55
Name: source, dtype: int64

In [24]:
df[df['count'] > 0].explode('raw_colocates')#.explode('raw_colocates')

,text,id,date,source,text_processed,raw_colocates,count,real_count
2,Ekologiczne i energooszczędne rozwiązania domi...,111,2021-02-21,DoRzeczy,ekologiczne i energooszczędne rozwiązania domi...,twierdzą niezbędne żeby przeciwstawić się zmi...,1,1
4,"Bjørn Lomborg przekłuwa balon paniki, nakręcan...",156,2021-02-28,DoRzeczy,bjrn lomborg przekłuwa balon paniki nakręcanej...,fałszywy alarm jak panika wokół zmiany klimatu...,7,7
4,"Bjørn Lomborg przekłuwa balon paniki, nakręcan...",156,2021-02-28,DoRzeczy,bjrn lomborg przekłuwa balon paniki nakręcanej...,klimatyści tak jak model kosztów zmian klimaty...,7,7
4,"Bjørn Lomborg przekłuwa balon paniki, nakręcan...",156,2021-02-28,DoRzeczy,bjrn lomborg przekłuwa balon paniki nakręcanej...,2030 r aby rozwiązać problem zmian klimatyczn...,7,7
4,"Bjørn Lomborg przekłuwa balon paniki, nakręcan...",156,2021-02-28,DoRzeczy,bjrn lomborg przekłuwa balon paniki nakręcanej...,trzeba by zrobić żeby utrzymać zmiany klimatyc...,7,7
...,...,...,...,...,...,...,...,...
7215,Dwoje dzieci naszego czytelnika pana Marcina j...,7215,2022-12-14,Wyborcza,dwoje dzieci naszego czytelnika pana marcina j...,ze stanem środowiska naturalnego więc zmiany k...,1,1
7219,Do końca roku pod głosowanie Rady Miejskiej w ...,7219,2022-12-22,Wyborcza,do końca roku pod głosowanie rady miejskiej w ...,sektorów i obszarów wrażliwych na zmiany klima...,1,1
7221,"- Zacznijmy lepiej od tego, że robiliśmy to w ...",7221,2022-12-24,Wyborcza,zacznijmy lepiej od tego że robiliśmy to w pe...,przyrody dlaczego dziś nie łapiemy zmian klima...,2,2
7221,"- Zacznijmy lepiej od tego, że robiliśmy to w ...",7221,2022-12-24,Wyborcza,zacznijmy lepiej od tego że robiliśmy to w pe...,pseudonauki z życia codziennego kwestionowanie...,2,2


## Saving

In [ ]:
df_exploded = df[df['count'] > 0].explode('raw_colocates')
df_exploded.reset_index(drop=True, inplace=True)
df_exploded.rename(columns={'raw_colocates': 'colocate', 'id': 'text_id'}, inplace=True)
df_exploded['id'] = df_exploded.index
df_exploded.drop(columns=['text', 'text_processed', 'count', 'real_count'], inplace=True)
df_exploded.head()

In [ ]:
df_exploded.to_csv("colocates.csv", index=False)